# Weights are subjective and often need adjustment
There are tools in `pyemu` to assist with weight adjustment to balance the contribution of various groups.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pyemu
import os, shutil
import sys
sys.path.append('..')

### Copy down the base  files for the freyberg model with K and R parameters

In [ ]:
import freyberg_setup as fs
fs.setup_pest_kr()
working_dir = fs.WORKING_DIR_KR
pst_name = fs.PST_NAME_KR


In [ ]:
fs.plot_model(working_dir, pst_name)

### We can read in the base PST file and do a quick run


In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))

In [ ]:
pst.control_data.noptmax

In [ ]:
pst.control_data.noptmax=0
pst.write(os.path.join(working_dir,pst_name))

In [ ]:
pyemu.os_utils.run("pestpp {0}".format(pst_name),cwd=working_dir)


In [ ]:
pst.phi

In [ ]:
pst.phi_components

In [ ]:
pst.phi_components_normalized

In [ ]:
pyemu.plot_utils.res_phi_pie(pst)

In [ ]:
pst.obs_groups

In [ ]:
groups = ['calflux','calhead']

In [ ]:
new_portions = [0.4,0.6]

In [ ]:
sum(new_portions)

In [ ]:
newcomponents = dict(zip(groups,[i*pst.phi for i in new_portions]))

In [ ]:
newcomponents

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,:]

In [ ]:
pst.phi_components

In [ ]:
pst.adjust_weights_by_list(['calflux'],1.0)
pst.phi_components

In [ ]:
pyemu.plot_utils.res_phi_pie(pst)

In [ ]:
pst.adjust_weights(obsgrp_dict=newcomponents)
pst.phi_components

In [ ]:
pyemu.plot_utils.res_phi_pie(pst)

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,:]

In [ ]:
pst.phi

In [ ]:
pst.write(os.path.join(working_dir,pst_name.replace(".pst","_rewght.pst")))
pyemu.os_utils.run("pestpp {0}".format(pst_name.replace(".pst","_rewght.pst")),cwd=working_dir)

## Relating weights to observation noise

Weights can initially be thought of as the reciprocal of observation noise, as characterized by standard deviation of the observation: 
$w=\frac{1}{\sigma}$

After adjustment, it's a good idea to calculate the equivalent $\sigma$ for each observation to determine whether it remains realistic. In this case:

In [ ]:
#initial weight for head
w=1
sig=1/w
sig

In [ ]:
#new weight for head
w=.774597
sig=1/w
sig

## This also applies after calibration, before uncertainty calculations...

According to the discrepancy prinicipal, the final objective function should equal the number of non-zero weighted observations....this is rarely the case.  This means we couldn't fit the data as well as we thought we could, which is usually a result of model error (not measurement error).  While this is an advanced topic, one really cheap way to cope with this problem is to scale the observation weights by the final residuals so that they reflect how well we actual could fit. 

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name.replace(".pst","_rewght.pst")))
pst.phi

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
len(pst.res.loc[pst.nnz_obs_names,:])

In [ ]:
pst.adjust_weights_resfile()
pst.phi

In [ ]:
pst.observation_data.loc[pst.nnz_obs_names,:]

See how much the weights were decreased?  This has implications for FOSM and Monte Carlo sampling...``pestpp`` does this rescaling automatically when it calculates FOSM uncertainty measures.  Its listed in the .rec file - check it out!